In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from typing import List, Any, Tuple
from datasets import Dataset
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from enum import Enum
import openai
import pickle
from InstructorEmbedding import INSTRUCTOR
import sentence_transformers
from datasets import load_dataset

In [13]:
query_data = load_dataset("princeton-nlp/LitSearch", "query", split="full")
corpus_clean_data = load_dataset("princeton-nlp/LitSearch", "corpus_clean", split="full")
corpus_s2orc_data = load_dataset("princeton-nlp/LitSearch", "corpus_s2orc", split="full")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

full-00000-of-00001.parquet:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

Generating full split:   0%|          | 0/597 [00:00<?, ? examples/s]

full-00000-of-00006.parquet:   0%|          | 0.00/275M [00:00<?, ?B/s]

...

full-00004-of-00008.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

full-00005-of-00008.parquet:   0%|          | 0.00/186M [00:00<?, ?B/s]

full-00006-of-00008.parquet:   0%|          | 0.00/186M [00:00<?, ?B/s]

full-00007-of-00008.parquet:   0%|          | 0.00/187M [00:00<?, ?B/s]

Generating full split:   0%|          | 0/64183 [00:00<?, ? examples/s]

In [14]:
query_df = query_data.to_pandas()
corpus_clean_df = corpus_clean_data.to_pandas()
corpus_s2orc_df = corpus_s2orc_data.to_pandas()

In [15]:
corpus_df = corpus_s2orc_df

In [18]:
pip install rank_bm25

In [22]:
from rank_bm25 import BM25Okapi
import pandas as pd
import ast

corpus_df["tokens"] = corpus_df["content"].astype(str).apply(lambda x: x.lower().split())

bm25 = BM25Okapi(corpus_df["tokens"].tolist())

In [26]:
def retrieve_documents(query, k=5):
    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)
    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
    return corpus_df.iloc[ranked_indices]["corpusid"].tolist()


retrieval_results = []

for _, row in query_df.iterrows():
    query_text = row["query"]
    relevant_ids = row["corpusids"]


    if isinstance(relevant_ids, str):
        relevant_ids = ast.literal_eval(relevant_ids)
    elif isinstance(relevant_ids, (list, tuple, set)):
        relevant_ids = list(relevant_ids)
    elif isinstance(relevant_ids, np.ndarray):
        relevant_ids = relevant_ids.tolist()
    else:
        relevant_ids = [relevant_ids]

    retrieved_ids = retrieve_documents(query_text, k=10)


    overlap = len(set(retrieved_ids) & set(map(int, relevant_ids)))

    retrieval_results.append({
        "query": query_text,
        "retrieved": retrieved_ids,
        "relevant": relevant_ids,
        "overlap": overlap
    })


results_df = pd.DataFrame(retrieval_results)
print(results_df.head())

                                               query  \
0  Are there any research papers on methods to co...   
1  Are there any resources available for translat...   
2  Are there any studies that explore post-hoc te...   
3  Are there any tools or studies that have focus...   
4  Are there papers that propose contextualized c...   

                                           retrieved                relevant  \
0  [222177226, 215238664, 258833648, 250390982, 2...             [202719327]   
1  [227231792, 252432736, 248780278, 21717882, 25...             [227231792]   
2  [253097869, 226254579, 218487034, 216869577, 2...  [226254579, 204976362]   
3  [18987788, 1612194, 15389538, 53473481, 671403...    [10961392, 12160022]   
4  [252917981, 244007367, 235669861, 236912915, 2...             [233296182]   

   overlap  
0        0  
1        1  
2        1  
3        0  
4        0  


In [29]:
import numpy as np

def precision_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    return len(set(retrieved_k) & set(relevant)) / k

def recall_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    return len(set(retrieved_k) & set(relevant)) / len(relevant) if len(relevant) > 0 else 0

def average_precision(retrieved, relevant):
    """ Computes average precision for a single query. """
    if not relevant:
        return 0.0

    precision_sum = 0
    relevant_retrieved = 0

    for i, doc in enumerate(retrieved):
        if doc in relevant:
            relevant_retrieved += 1
            precision_sum += relevant_retrieved / (i + 1)
    return precision_sum / len(relevant) if relevant else 0.0

def mean_average_precision(retrieved_list, relevant_list):
    """ Computes MAP across multiple queries. """
    return np.mean([average_precision(retrieved, relevant) for retrieved, relevant in zip(retrieved_list, relevant_list)])

def reciprocal_rank(retrieved, relevant):
    """ Computes reciprocal rank for a single query. """
    for i, doc in enumerate(retrieved):
        if doc in relevant:
            return 1 / (i + 1)
    return 0

def mean_reciprocal_rank(retrieved_list, relevant_list):
    """ Computes MRR across multiple queries. """
    return np.mean([reciprocal_rank(retrieved, relevant) for retrieved, relevant in zip(retrieved_list, relevant_list)])


def dcg_at_k(retrieved, relevant, k):
    """ Computes Discounted Cumulative Gain """
    retrieved_k = retrieved[:k]
    return sum((1 / np.log2(i + 2)) if doc in relevant else 0 for i, doc in enumerate(retrieved_k))

def ndcg_at_k(retrieved, relevant, k):
    """ Computes Normalized DCG """
    ideal_dcg = sum((1 / np.log2(i + 2)) for i in range(min(k, len(relevant))))  # Ideal ranking
    return dcg_at_k(retrieved, relevant, k) / ideal_dcg if ideal_dcg > 0 else 0


In [31]:
retrieval_results = []

for _, row in query_df.iterrows():
    query_text = row["query"]

    relevant_ids = row["corpusids"]
    if isinstance(relevant_ids, str):
        relevant_ids = ast.literal_eval(relevant_ids)
    elif isinstance(relevant_ids, np.ndarray):
        relevant_ids = relevant_ids.tolist()
    elif not isinstance(relevant_ids, list):
        relevant_ids = [relevant_ids]

    retrieved_ids = retrieve_documents(query_text, k=10)


    retrieval_results.append({
        "query": query_text,
        "retrieved": retrieved_ids,
        "relevant": relevant_ids,
        "Precision@5": precision_at_k(retrieved_ids, relevant_ids, 5),
        "Precision@10": precision_at_k(retrieved_ids, relevant_ids, 10),
        "Recall@5": recall_at_k(retrieved_ids, relevant_ids, 5),
        "Recall@10": recall_at_k(retrieved_ids, relevant_ids, 10),
        "AP": average_precision(retrieved_ids, relevant_ids),
        "MRR": reciprocal_rank(retrieved_ids, relevant_ids),
        "NDCG@5": ndcg_at_k(retrieved_ids, relevant_ids, 5),
        "NDCG@10": ndcg_at_k(retrieved_ids, relevant_ids, 10),
    })

results_df = pd.DataFrame(retrieval_results)
print(results_df.head())


print(f"MAP: {mean_average_precision(results_df['retrieved'], results_df['relevant'])}")
print(f"MRR: {mean_reciprocal_rank(results_df['retrieved'], results_df['relevant'])}")


                                               query  \
0  Are there any research papers on methods to co...   
1  Are there any resources available for translat...   
2  Are there any studies that explore post-hoc te...   
3  Are there any tools or studies that have focus...   
4  Are there papers that propose contextualized c...   

                                           retrieved                relevant  \
0  [222177226, 215238664, 258833648, 250390982, 2...             [202719327]   
1  [227231792, 252432736, 248780278, 21717882, 25...             [227231792]   
2  [253097869, 226254579, 218487034, 216869577, 2...  [226254579, 204976362]   
3  [18987788, 1612194, 15389538, 53473481, 671403...    [10961392, 12160022]   
4  [252917981, 244007367, 235669861, 236912915, 2...             [233296182]   

   Precision@5  Precision@10  Recall@5  Recall@10    AP  MRR    NDCG@5  \
0          0.0           0.0       0.0        0.0  0.00  0.0  0.000000   
1          0.2           0.1      

In [32]:
results_df.head(50)

,query,retrieved,relevant,Precision@5,Precision@10,Recall@5,Recall@10,AP,MRR,NDCG@5,NDCG@10
0,Are there any research papers on methods to co...,"[222177226, 215238664, 258833648, 250390982, 2...",[202719327],0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Are there any resources available for translat...,"[227231792, 252432736, 248780278, 21717882, 25...",[227231792],0.2,0.1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,Are there any studies that explore post-hoc te...,"[253097869, 226254579, 218487034, 216869577, 2...","[226254579, 204976362]",0.2,0.1,0.500000,0.500000,0.250000,0.500000,0.386853,0.386853
3,Are there any tools or studies that have focus...,"[18987788, 1612194, 15389538, 53473481, 671403...","[10961392, 12160022]",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Are there papers that propose contextualized c...,"[252917981, 244007367, 235669861, 236912915, 2...",[233296182],0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Are there studies that combine convolutional a...,"[248218685, 14142685, 225063037, 226283822, 26...",[17297069],0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Can you direct me to research that explores me...,"[247518809, 251718892, 211258645, 236477336, 1...",[174801080],0.2,0.1,1.000000,1.000000,0.200000,0.200000,0.386853,0.386853
7,Can you direct me to studies that explore tech...,"[259376736, 259376778, 259376471, 259376811, 2...",[247593812],0.0,0.1,0.000000,1.000000,0.100000,0.100000,0.000000,0.289065
8,Can you list some publications that discuss th...,"[252846088, 198978291, 220445728, 17500487, 21...","[561429, 17553490, 16509032]",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Can you point me to a paper that discussed tra...,"[4567927, 259095915, 116084478, 62354584, 2135...",[201646309],0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
qasper_dataset = load_dataset("allenai/qasper")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.64k [00:00<?, ?B/s]

qasper.py:   0%|          | 0.00/5.95k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/14.4M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/4.75M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/7.07M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/888 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/281 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/416 [00:00<?, ? examples/s]

In [8]:
train_data = qasper_dataset["train"]
validation_data = qasper_dataset["validation"]
test_data = qasper_dataset["test"]

In [9]:
train_df = pd.DataFrame(train_data)
validation_df = pd.DataFrame(validation_data)
test_df = pd.DataFrame(test_data)

In [11]:
total_rows = train_df.shape[0] + validation_df.shape[0] + test_df.shape[0]
print(f"Total number of rows in QASPER dataset: {total_rows}")


Total number of rows in QASPER dataset: 1585


In [12]:
dataset = load_dataset("allenai/sciq")

README.md:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.99M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/339k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
train_df = pd.DataFrame(dataset["train"])
validation_df = pd.DataFrame(dataset["validation"])
test_df = pd.DataFrame(dataset["test"])

In [15]:
train_df.head()

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...


In [16]:
# Print the size of each split
for split in dataset.keys():
    print(f"{split} size: {len(dataset[split])}")


train size: 11679
validation size: 1000
test size: 1000
